PRÁCTICA 1 - TIPOLOGÍA Y CICLO DE VIDA DE LOS DATOS

Caso práctico orientado a aprender a identificar los datos relevantes para un proyecto analítico y usar herramientas de extracción de datos. 

El objetivo de esta actividad será la creación de un dataset a partir de los datos contenidos en un sitio web. 

In [1]:
import requests
import csv
import pandas
import numpy as np

In [2]:
from bs4 import BeautifulSoup

In [3]:
# Función descargaPaginaWeb que permite recuperar la información correspondiente a la respuesta de la petición. 
# url: corresponde con la url del sitio donde se hará el web scraping
def descargaPaginaWeb(url):
    respuesta = requests.get(
        url,
        #headers = {
        #    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:93.0) Gecko/20100101 Firefox/93.0'
        #}
    )
    return respuesta

# Función extraerImagen que extrae la url de una imagen contenida en otra url de información sobre un sismo sobre la que
# hay que hacer web scraping. La función retorna la url de la imagen o NA si no hay información de la imagen.
# tag: etiqueta html
# parametro: parametro que se añade a la etiqueta
def extraerImagen(tag, parametro):
    info_form = tag.find('form')
    output = info_form['action'] + '?evid=' + parametro
    pagina = descargaPaginaWeb(output)
    contenido_web = BeautifulSoup(pagina.text, 'html.parser')
    imagen = contenido_web.find('img', attrs={"alt": "Foto Detalle"})
    if type(imagen['src']) != 'str':
        return 'NA'
    else:
        return imagen['src']
    
        
# Función extraerDatos que extrae de una página web la información requerida. La función retorna una lista con la 
# información de todos los sismos producidos (cada sismo se almacena en un diccionario con los nombres de los campos y 
# sus valores).
# respuesta: respuesta obtenida al preguntar a una url específica
def extraerDatos(respuesta):
    contenido_web  = BeautifulSoup(respuesta.text, 'html.parser')
    sismos = []
    nombre_columnas = []
    tabla = contenido_web .find_all('table')[0]
    filas = tabla.find_all('tr')
    for fila in filas:
        cols = fila.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        nom_cols = fila.find_all('th')
        nom_cols = [ele.text.strip() for ele in nom_cols]
        if len(nom_cols) > 0:
            nombre_columnas = nom_cols
        if len(cols) > 0:
            sismos.append(cols) 

    sismos_df = pandas.DataFrame(sismos,columns=nombre_columnas)
    return sismos_df
    
# Función convertirDatos que convierte los datos de str al tipo correspondiente.
# dataframe
# nombreArchivo: nombre del archivo csv
def convertirDatos(dataFrame):
    dataFrame['Fecha'] = pandas.to_datetime(data_frame['Fecha'], format='%d/%m/%Y')
    dataFrame['Hora UTC'] = pandas.to_datetime(data_frame['Hora UTC'], format='%H:%M:%S')
    dataFrame['Hora Local(*)'] = pandas.to_datetime(data_frame['Hora Local(*)'], format='%H:%M:%S')
    dataFrame['Profundidad(km)'] = pandas.to_numeric(data_frame['Profundidad(km)'])
    dataFrame['Magnitud'] = pandas.to_numeric(data_frame['Magnitud'])
    dataFrame['Latitud'] = pandas.to_numeric(data_frame['Latitud'])
    dataFrame['Longitud'] = pandas.to_numeric(data_frame['Longitud'])
    
# Función crearArchivoCSVDesdeLista que crea un archivo csv a partir de una lista de diccionarios.
# lista: lista con los diccionarios
# nombreArchivo: nombre del archivo csv
def crearArchivoCSVDesdeLista(dataFrame, nombreArchivo):
    dataFrame.to_csv(nombreArchivo,index=False)


# Función principal
def main():
    respuesta = descargaPaginaWeb('https://www.ign.es/web/ign/portal/ultimos-terremotos/-/ultimos-terremotos/get30dias')
    lista = extraerDatos(respuesta)
    crearArchivoCSVDesdeLista(lista, 'dataset.csv')

In [4]:
main()